## First Draft
Just basic setup

## Second Draft
Appropriate Prompting

## Third Draft
Shuffled Dataset

# Setup

In [ ]:
!pip install transformers torch datasets evaluate

In [ ]:
import random
import math
from transformers import BertForMaskedLM, BertTokenizer, RobertaForMaskedLM, RobertaTokenizer, DistilBertForMaskedLM, DistilBertTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch
import json
from datasets import Dataset
import evaluate
import os
import numpy as np
import shutil
import copy

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import userdata
# os.environ["HUGGINGFACE_API_TOKEN"] = userdata.get('BTP_HF_TOKEN')

# Model choices

In [ ]:
bert_base = "google-bert/bert-base-uncased"
roberta_base = "FacebookAI/roberta-base"
distilbert_base = "distilbert/distilbert-base-uncased"

In [ ]:
bert_model = BertForMaskedLM.from_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/first_draft/bert_trained_model_draft1")
bert_tokenizer = BertTokenizer.from_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/first_draft/bert_trained_model_draft1")

# roberta_model = RobertaForMaskedLM.from_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/first_draft/roberta_trained_model_draft1")
# roberta_tokenizer = RobertaTokenizer.from_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/first_draft/roberta_trained_model_draft1")

# distilbert_model = DistilBertForMaskedLM.from_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/first_draft/distilbert_trained_model_draft1")
# distilbert_tokenizer = DistilBertTokenizer.from_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/first_draft/distilbert_trained_model_draft1")

In [ ]:
# bert_model = BertForMaskedLM.from_pretrained("google-bert/bert-base-uncased")
# bert_tokenizer = BertTokenizer.from_pretrained("google-bert/bert-base-uncased")

# roberta_model = RobertaForMaskedLM.from_pretrained("FacebookAI/roberta-base")
# roberta_tokenizer = RobertaTokenizer.from_pretrained("FacebookAI/roberta-base")

# distilbert_model = DistilBertForMaskedLM.from_pretrained("distilbert/distilbert-base-uncased")
# distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [ ]:
bert_model = bert_model.to("cuda")
# roberta_model = roberta_model.to("cuda")
# distilbert_model = distilbert_model.to("cuda")
bert_data_collator = DataCollatorForLanguageModeling(tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15)
# roberta_data_collator = DataCollatorForLanguageModeling(tokenizer=roberta_tokenizer, mlm=True, mlm_probability=0.15)
# distilbert_data_collator = DataCollatorForLanguageModeling(tokenizer=distilbert_tokenizer, mlm=True, mlm_probability=0.15)

In [ ]:
def mask_model_language(model, tokenizer):
  def masked_prediction(sentence, top_k=5):
    # Tokenize the input text and identify the masked token's position
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    # Get model outputs
    with torch.no_grad():
        outputs = model(**inputs)

    # The prediction logits for the masked tokens
    logits = outputs.logits

    # Get the predictions for the masked position
    mask_token_logits = logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, top_k, dim=1).indices[0].tolist()

    # Convert token ids to words
    predicted_words = [tokenizer.decode([token]) for token in top_5_tokens]

    return predicted_words
  return masked_prediction

In [ ]:
bert_mask_prediction = mask_model_language(bert_model, bert_tokenizer)
# roberta_mask_prediction = mask_model_language(roberta_model, roberta_tokenizer)
# distilbert_mask_prediction = mask_model_language(distilbert_model, distilbert_tokenizer)

In [ ]:
def model_prediction(model, tokenizer):
    def predict(sentence):
        inputs = tokenizer(sentence, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs).logits
        return [tokenizer.decode([token]) for token in torch.topk(outputs[0, :, :], len(outputs[0, :, :]), dim=1).indices[0].tolist()]
    return predict

In [ ]:
bert_base_prediction = model_prediction(bert_model, bert_tokenizer)
# roberta_base_prediction = model_prediction(roberta_model, roberta_tokenizer)
# distilbert_base_prediction = model_prediction(distilbert_model, distilbert_tokenizer)

# Dataset setup

In [ ]:
system_prompt = """I will be giving a sentence of the current event, its next event, its occurence time and its duration. One of these values would be masked, your task is to predict the masked token\n"""

In [ ]:
with open("/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/masked_dataset.json", "r") as f:
  masked_dataset = json.load(f)

In [ ]:
with open("/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/temp_temporal_graph_data1.json", "r") as f:
  new_dataset = json.load(f)

In [ ]:
bert_masked = masked_dataset["bert_masked"]
random.shuffle(bert_masked)
# roberta_masked = masked_dataset["roberta_masked"]
# random.shuffle(roberta_masked)
# distilbert_masked = copy.deepcopy(masked_dataset["bert_masked"])
# random.shuffle(distilbert_masked)

In [ ]:
train_bert_masked = bert_masked[:math.ceil(0.75 * len(bert_masked))]
valid_bert_masked = bert_masked[math.ceil(0.75 * len(bert_masked)):math.ceil(0.75 * len(bert_masked)) + math.ceil(0.10 * len(bert_masked))]
test_bert_masked = bert_masked[math.ceil(0.75 * len(bert_masked)) + math.ceil(0.10 * len(bert_masked)):]

# train_roberta_masked = roberta_masked[:math.ceil(0.75 * len(roberta_masked))]
# valid_roberta_masked = roberta_masked[math.ceil(0.75 * len(roberta_masked)):math.ceil(0.75 * len(roberta_masked)) + math.ceil(0.10 * len(roberta_masked))]
# test_roberta_masked = roberta_masked[math.ceil(0.75 * len(roberta_masked)) + math.ceil(0.10 * len(roberta_masked)):]

# train_distilbert_masked = distilbert_masked[:math.ceil(0.75 * len(distilbert_masked))]
# valid_distilbert_masked = distilbert_masked[math.ceil(0.75 * len(distilbert_masked)):math.ceil(0.75 * len(distilbert_masked)) + math.ceil(0.10 * len(distilbert_masked))]
# test_distilbert_masked = distilbert_masked[math.ceil(0.75 * len(distilbert_masked)) + math.ceil(0.10 * len(distilbert_masked)):]

In [ ]:
train_bert_masked_dataset = Dataset.from_list(train_bert_masked)
valid_bert_masked_dataset = Dataset.from_list(valid_bert_masked)
test_bert_masked_dataset = Dataset.from_list(test_bert_masked)

# train_roberta_masked_dataset = Dataset.from_list(train_roberta_masked)
# valid_roberta_masked_dataset = Dataset.from_list(valid_roberta_masked)
# test_roberta_masked_dataset = Dataset.from_list(test_roberta_masked)

# train_distilbert_masked_dataset = Dataset.from_list(train_distilbert_masked)
# valid_distilbert_masked_dataset = Dataset.from_list(valid_distilbert_masked)
# test_distilbert_masked_dataset = Dataset.from_list(test_distilbert_masked)

In [ ]:
def f(example, tokenizer):
    sentence = system_prompt + example['sentence']
    label = example['mask_value']

    # Tokenize sentence and label, ensuring flat structure
    inputs = tokenizer(sentence, truncation=True, padding='max_length', max_length=512, return_tensors=None)
    labels = tokenizer(label, truncation=True, padding='max_length', max_length=512, return_tensors=None)["input_ids"]

    # Add the labels as a flat list of token IDs
    inputs['labels'] = labels

    return inputs

In [ ]:
train_bert_masked_dataset1 = train_bert_masked_dataset.map(lambda x: f(x, bert_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])
valid_bert_masked_dataset1 = valid_bert_masked_dataset.map(lambda x: f(x, bert_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])
test_bert_masked_dataset1 = test_bert_masked_dataset.map(lambda x: f(x, bert_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])

# train_roberta_masked_dataset1 = train_roberta_masked_dataset.map(lambda x: f(x, roberta_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])
# valid_roberta_masked_dataset1 = valid_roberta_masked_dataset.map(lambda x: f(x, roberta_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])
# test_roberta_masked_dataset1 = test_roberta_masked_dataset.map(lambda x: f(x, roberta_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])

# train_distilbert_masked_dataset1 = train_distilbert_masked_dataset.map(lambda x: f(x, distilbert_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])
# valid_distilbert_masked_dataset1 = valid_distilbert_masked_dataset.map(lambda x: f(x, distilbert_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])
# test_distilbert_masked_dataset1 = test_distilbert_masked_dataset.map(lambda x: f(x, distilbert_tokenizer), batched=False).remove_columns(['sentence', 'mask_value'])

Map:   0%|          | 0/1614 [00:00<?, ? examples/s]

Map:   0%|          | 0/216 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

# Accuracy

In [ ]:
accuracy_metric = evaluate.load("accuracy")

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)  # Get predicted token IDs

    # Only consider non-padding tokens in labels
    mask = labels != -100  # Mask for ignoring -100 labels

    # Calculate accuracy only where mask is True
    accuracy = accuracy_metric.compute(predictions=predictions[mask], references=labels[mask])
    return accuracy

# Training

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    remove_unused_columns=False,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize the Trainer
bert_trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_bert_masked_dataset1,
    eval_dataset=valid_bert_masked_dataset1,
    data_collator=bert_data_collator,
    compute_metrics=compute_accuracy  # Pass custom metric function
)

# roberta_trainer = Trainer(
#     model=roberta_model,
#     args=training_args,
#     train_dataset=train_roberta_masked_dataset1,
#     eval_dataset=valid_roberta_masked_dataset1,
#     data_collator=roberta_data_collator,
#     compute_metrics=compute_accuracy  # Pass custom metric function
# )

# distilbert_trainer = Trainer(
#     model=distilbert_model,
#     args=training_args,
#     train_dataset=train_distilbert_masked_dataset1,
#     eval_dataset=valid_distilbert_masked_dataset1,
#     data_collator=distilbert_data_collator,
#     compute_metrics=compute_accuracy  # Pass custom metric function
# )

In [ ]:
bert_trainer.train()
roberta_trainer.train()
distilbert_trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.612800,0.519147
2,0.418200,0.343464
3,0.415000,0.349412


Epoch,Training Loss,Validation Loss
1,0.509400,0.451250
2,0.349300,0.254422
3,0.310900,0.174481


Epoch,Training Loss,Validation Loss
1,0.581700,0.539111
2,0.390700,0.402517
3,0.334000,0.356099


TrainOutput(global_step=606, training_loss=0.5849329764300054, metrics={'train_runtime': 353.901, 'train_samples_per_second': 13.682, 'train_steps_per_second': 1.712, 'total_flos': 641861117816832.0, 'train_loss': 0.5849329764300054, 'epoch': 3.0})

In [ ]:
bert_trainer.save_model("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/bert_trained_model_draft1")
bert_tokenizer.save_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/bert_trained_model_draft1")

roberta_trainer.save_model("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/roberta_trained_model_draft1")
roberta_tokenizer.save_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/roberta_trained_model_draft1")

distilbert_trainer.save_model("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/distilbert_trained_model_draft1")
distilbert_tokenizer.save_pretrained("/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/distilbert_trained_model_draft1")

('/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/distilbert_trained_model_draft1/tokenizer_config.json',
 '/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/distilbert_trained_model_draft1/special_tokens_map.json',
 '/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/distilbert_trained_model_draft1/vocab.txt',
 '/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/distilbert_trained_model_draft1/added_tokens.json')

In [ ]:
shutil.move("/content/logs", "/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/logs")
shutil.move("/content/results", "/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/results")

'/content/drive/MyDrive/Temporal Chatbot BTP/mlm models/third_draft/results'

# Testing

In [ ]:
# Evaluate on the test dataset
bert_test_results = bert_trainer.evaluate(test_bert_masked_dataset1)
roberta_test_results = roberta_trainer.evaluate(test_roberta_masked_dataset1)
distilbert_test_results = distilbert_trainer.evaluate(test_distilbert_masked_dataset1)

print("Bert Test Results:", bert_test_results)
print("RoBerta Test Results:", roberta_test_results)
print("DistilBert Test Results:", distilbert_test_results)

Bert Test Results: {'eval_loss': 0.26024556159973145, 'eval_runtime': 11.479, 'eval_samples_per_second': 28.051, 'eval_steps_per_second': 3.572, 'epoch': 3.0}
RoBerta Test Results: {'eval_loss': 0.22452180087566376, 'eval_runtime': 12.9965, 'eval_samples_per_second': 24.776, 'eval_steps_per_second': 3.155, 'epoch': 3.0}
DistilBert Test Results: {'eval_loss': 0.3959859609603882, 'eval_runtime': 7.7461, 'eval_samples_per_second': 41.57, 'eval_steps_per_second': 5.293, 'epoch': 3.0}


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
bert_eval_results = bert_trainer.evaluate()
# roberta_eval_results = roberta_trainer.evaluate()
# distilbert_eval_results = distilbert_trainer.evaluate()

print(f"Validation Accuracy: {bert_eval_results['eval_accuracy']:.4f}")
# print(f"Validation Accuracy: {roberta_eval_results['eval_accuracy']:.4f}")
# print(f"Validation Accuracy: {distilbert_eval_results['eval_accuracy']:.4f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.79 GiB. GPU 0 has a total capacity of 14.75 GiB of which 2.05 GiB is free. Process 3665 has 12.70 GiB memory in use. Of the allocated memory 8.80 GiB is allocated by PyTorch, and 3.77 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Preparing of dataset

In [ ]:
# dataset_path1 = "/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/temporal_ordering_changed2.json"
# dataset_path = "/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/temp_temporal_graph_data.json"
# dataset_path2 = "/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/temporal_ordering_split_changed.json"

In [ ]:
# with open(dataset_path, "r") as f:
#   dataset = json.load(f)

In [ ]:
# new_dataset = dict()
# for i in dataset:
#     new_dataset[i] = dict()
#     for j in dataset[i]:
#         new_dataset[i][j] = dataset[i][j] if j == 'ngbs' else {"time_value": dataset[i][j][0], "time_duration": dataset[i][j][1]}

In [ ]:
# with open("/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/temp_temporal_graph_data1.json", "w") as f:
#   json.dump(new_dataset, f)

In [ ]:
# with open('/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/temp_temporal_graph_data1.json', 'r') as f:
#     new_dataset = json.load(f)

In [ ]:
# lines = []
# for current_event in new_dataset:
#     current_line = f"{current_event} -> "
#     next_events = new_dataset[current_event]['ngbs']
#     for next_event in next_events:
#         current_line += f"{next_event}, "
#     for time_values in new_dataset[current_event]['time_point']:
#         if time_values == 'time_value':
#             current_line += f": Time value = {new_dataset[current_event]['time_point'][time_values][1]} "
#         else:
#             current_line += f": Time duration = {new_dataset[current_event]['time_point'][time_values][1]} "
#     lines.append(current_line)

current event , next event, time point, time duration

In [ ]:
# bert_masked_curr_event_lines = []
# for current_event in new_dataset:
#     mask_value = current_event
#     current_line = f"Current Event: [MASK]; "

#     time_value = new_dataset[current_event]['time_point']['time_value'][1]
#     current_line += f"Time point: {time_value} minutes; "

#     time_duration = new_dataset[current_event]['time_point']['time_duration'][1]
#     current_line += f"Time duration: {time_duration} minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         new_line = current_line + f"Next event: {next_event}."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         bert_masked_curr_event_lines.append(current_data)

# bert_masked_time_values_lines = []
# for current_event in new_dataset:
#     current_line = f"Current Event: {current_event}; "

#     mask_value = str(new_dataset[current_event]['time_point']['time_value'][1])
#     current_line += f"Time point: [MASK] minutes; "

#     time_duration = new_dataset[current_event]['time_point']['time_duration'][1]
#     current_line += f"Time duration: {time_duration} minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         new_line = current_line + f"Next event: {next_event}."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         bert_masked_time_values_lines.append(current_data)

# bert_masked_time_duration_lines = []
# for current_event in new_dataset:
#     current_line = f"Current Event: {current_event}; "

#     time_value = new_dataset[current_event]['time_point']['time_value'][1]
#     current_line += f"Time point: {time_value} minutes; "

#     mask_value = str(new_dataset[current_event]['time_point']['time_duration'][1])
#     current_line += f"Time duration: [MASK] minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         new_line = current_line + f"Next event: {next_event}."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         bert_masked_time_duration_lines.append(current_data)

# bert_masked_next_event_lines = []
# for current_event in new_dataset:
#     current_line = f"Current Event: {current_event}; "

#     time_value = new_dataset[current_event]['time_point']['time_value'][1]
#     current_line += f"Time point: {time_value} minutes; "

#     time_duration = new_dataset[current_event]['time_point']['time_duration'][1]
#     current_line += f"Time duration: {time_duration} minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         mask_value = next_event
#         new_line = current_line + f"Next event: [MASK]."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         bert_masked_next_event_lines.append(current_data)

In [ ]:
# roberta_masked_curr_event_lines = []
# for current_event in new_dataset:
#     mask_value = current_event
#     current_line = f"Current Event: <mask>; "

#     time_value = new_dataset[current_event]['time_point']['time_value'][1]
#     current_line += f"Time point: {time_value} minutes; "

#     time_duration = new_dataset[current_event]['time_point']['time_duration'][1]
#     current_line += f"Time duration: {time_duration} minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         new_line = current_line + f"Next event: {next_event}."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         roberta_masked_curr_event_lines.append(current_data)

# roberta_masked_time_values_lines = []
# for current_event in new_dataset:
#     current_line = f"Current Event: {current_event}; "

#     mask_value = str(new_dataset[current_event]['time_point']['time_value'][1])
#     current_line += f"Time point: <mask> minutes; "

#     time_duration = new_dataset[current_event]['time_point']['time_duration'][1]
#     current_line += f"Time duration: {time_duration} minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         new_line = current_line + f"Next event: {next_event}."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         roberta_masked_time_values_lines.append(current_data)

# roberta_masked_time_duration_lines = []
# for current_event in new_dataset:
#     current_line = f"Current Event: {current_event}; "

#     time_value = new_dataset[current_event]['time_point']['time_value'][1]
#     current_line += f"Time point: {time_value} minutes; "

#     mask_value = str(new_dataset[current_event]['time_point']['time_duration'][1])
#     current_line += f"Time duration: <mask> minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         new_line = current_line + f"Next event: {next_event}."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         roberta_masked_time_duration_lines.append(current_data)

# roberta_masked_next_event_lines = []
# for current_event in new_dataset:
#     current_line = f"Current Event: {current_event}; "

#     time_value = new_dataset[current_event]['time_point']['time_value'][1]
#     current_line += f"Time point: {time_value} minutes; "

#     time_duration = new_dataset[current_event]['time_point']['time_duration'][1]
#     current_line += f"Time duration: {time_duration} minutes; "

#     for next_event in new_dataset[current_event]['ngbs']:
#         mask_value = next_event
#         new_line = current_line + f"Next event: <mask>."
#         current_data = {
#             "sentence": new_line,
#             "mask_value": mask_value
#         }
#         roberta_masked_next_event_lines.append(current_data)

In [ ]:
# bert_masked_lines = bert_masked_curr_event_lines + bert_masked_time_values_lines + bert_masked_time_duration_lines + bert_masked_next_event_lines
# roberta_masked_lines = roberta_masked_curr_event_lines + roberta_masked_time_values_lines + roberta_masked_time_duration_lines + roberta_masked_next_event_lines

In [ ]:
# masked_dataset = {
#     "bert_masked": bert_masked_lines,
#     "roberta_masked": roberta_masked_lines
# }

In [ ]:
# with open("/content/drive/MyDrive/Temporal Chatbot BTP/BTP Datasets/masked_dataset.json", "w") as f:
#   json.dump(masked_dataset, f)